# Data Exploration
This is notebook focusing on understanding EHR and DICOM data structure

## Adjust data
This section of the notebook focuses on adjusting and transforming the data from various FHIR resources. The data is loaded, flattened, and normalized to ensure consistency and ease of analysis.

In [59]:
import os
import dotenv

dotenv.load_dotenv('../.env')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [60]:
dpath = os.getenv('EHR_PATH')

# # Get the first of first directory in EHR data
# first_dir = os.path.join(dpath, os.listdir(dpath)[0])
# test_dir = os.path.join(first_dir, os.listdir(first_dir)[0])
# items_in_test_dir = os.listdir(test_dir)

# Get all directories in first dir
first_dir = os.path.join(dpath, os.listdir(dpath)[0])
items_in_test_dir = []
for dir in os.listdir(first_dir):
    dipath = os.path.join(first_dir, dir)
    items_in_dir = [os.path.join(dipath, item) for item in os.listdir(dipath)]
    items_in_test_dir.extend(items_in_dir)

len(items_in_test_dir)


531

In [61]:
# Create meta data frame
meta_df = pd.DataFrame(items_in_test_dir, columns=['path'])

def get_meta_data(row):
    path = row['path']
    with open(path) as f:
        data = json.load(f)
    row['data'] = data
    return row

meta_df = meta_df.apply(get_meta_data, axis=1)

# Print the keys of the first data
print(meta_df['data'][0].keys())

dict_keys(['type', 'entry', 'resourceType'])


In [62]:
def flatten_data(row):
    """
    Flatten the data dictionary into the row
    """
    data = row['data']
    for key in data.keys():
        row[key] = data[key]
    return row

In [63]:
def flatten_object(row):
    for field in row:
        print(field)
        if isinstance(field, (dict, object)):
            try:
                for key in row[field]:
                    row[field + '_' + key] = row[field][key]
                del row[field]
            except:
                pass

In [64]:
meta_df = meta_df.apply(flatten_data, axis=1)

meta_df.drop(columns=['data'], inplace=True)
meta_df.head()

,path,type,entry,resourceType
0,E:\Thesis\project\.data\fhir\00\000\0000e4c0-2...,collection,[{'fullUrl': 'urn:uuid:80bb41d3-ebb4-42e6-b087...,Bundle
1,E:\Thesis\project\.data\fhir\00\000\000148c5-9...,collection,[{'fullUrl': 'urn:uuid:bb357dc7-2f8a-444e-ba57...,Bundle
2,E:\Thesis\project\.data\fhir\00\000\00023d13-0...,collection,[{'fullUrl': 'urn:uuid:c71d235f-9b58-4106-b744...,Bundle
3,E:\Thesis\project\.data\fhir\00\000\0002bd05-b...,collection,[{'fullUrl': 'urn:uuid:509474a0-baf3-48f2-a595...,Bundle
4,E:\Thesis\project\.data\fhir\00\000\0003292a-9...,collection,[{'fullUrl': 'urn:uuid:f10fd607-58ce-4be6-b1bd...,Bundle


In [65]:
# Explode the entry column
all_df = meta_df.copy().explode('entry').reset_index(drop=True)

all_df.rename(columns={'entry': 'data'}, inplace=True)
all_df = all_df.apply(flatten_data, axis=1)
all_df.drop(columns=['resourceType', 'type', 'data'], inplace=True)
all_df['resourceType'] = all_df['resource'].apply(lambda x: x['resourceType'])
all_df.head()

,fullUrl,path,resource,resourceType
0,urn:uuid:80bb41d3-ebb4-42e6-b087-6ee89b0463a9,E:\Thesis\project\.data\fhir\00\000\0000e4c0-2...,"{'id': '80bb41d3-ebb4-42e6-b087-6ee89b0463a9',...",Patient
1,urn:uuid:2c722ad1-5a59-4f6a-90cc-c55f86bbd9c6,E:\Thesis\project\.data\fhir\00\000\0000e4c0-2...,"{'id': '2c722ad1-5a59-4f6a-90cc-c55f86bbd9c6',...",Encounter
2,urn:uuid:593c21c4-8cce-47cd-bd70-41e4a7b25557,E:\Thesis\project\.data\fhir\00\000\0000e4c0-2...,"{'id': '593c21c4-8cce-47cd-bd70-41e4a7b25557',...",Condition
3,urn:uuid:1f954180-ec49-455a-95aa-3b7414ffeab6,E:\Thesis\project\.data\fhir\00\000\0000e4c0-2...,"{'id': '1f954180-ec49-455a-95aa-3b7414ffeab6',...",Observation
4,urn:uuid:556966e3-42a8-4359-838a-8771dd2a2a23,E:\Thesis\project\.data\fhir\00\000\0000e4c0-2...,"{'id': '556966e3-42a8-4359-838a-8771dd2a2a23',...",Observation


In [66]:
# Get the unique resource types
all_df['resourceType'].unique()

array(['Patient', 'Encounter', 'Condition', 'Observation', 'Immunization',
       'CarePlan', 'Procedure', 'DiagnosticReport', 'MedicationRequest',
       'AllergyIntolerance'], dtype=object)

In [67]:
from typing import Dict

detail_dfs: Dict[str, pd.DataFrame] = {
    resource_type: all_df.copy()[all_df['resourceType'] == resource_type]
    for resource_type in all_df['resourceType'].unique()
}
for df in detail_dfs:
    detail_dfs[df].drop(columns=['fullUrl', 'path'], inplace=True)
    detail_dfs[df].rename(columns={'resource': 'data'}, inplace=True)
    normalized_data = pd.json_normalize(detail_dfs[df]['data'])
    detail_dfs[df] = pd.concat([detail_dfs[df].drop(columns=['data']), normalized_data], axis=1)
    print(f"\nResource type: {df}")
    detail_dfs[df].info(show_counts=False, memory_usage=False)


Resource type: Patient
<class 'pandas.core.frame.DataFrame'>
Index: 1054 entries, 0 to 530
Data columns (total 17 columns):
 #   Column                Dtype  
---  ------                -----  
 0   resourceType          object 
 1   id                    object 
 2   extension             object 
 3   identifier            object 
 4   name                  object 
 5   telecom               object 
 6   gender                object 
 7   birthDate             object 
 8   address               object 
 9   multipleBirthBoolean  object 
 10  photo                 object 
 11  resourceType          object 
 12  text.status           object 
 13  text.div              object 
 14  maritalStatus.coding  object 
 15  deceasedDateTime      object 
 16  multipleBirthInteger  float64
dtypes: float64(1), object(16)
Resource type: Encounter
<class 'pandas.core.frame.DataFrame'>
Index: 9472 entries, 1 to 5101
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------          

If the field in the dataframe is a list, it should be stored in a new sub dataframe.

In [68]:
def explode_df(row, df):
    """
    This function separates the list columns into separate dataframes from the dataframe with ID
    """
    cols_to_remove = []
    cols = detail_dfs[df].columns
    try:
        id = row['id']
    except:
        return cols_to_remove
    for col in cols:
        if isinstance(row[col], list):
            new_df = f'{df}.{col}'
            if new_df not in detail_dfs:
                detail_dfs[new_df] = pd.DataFrame()
            new_row = pd.DataFrame(row[col])
            new_row[f'{df}_id'] = id
            detail_dfs[new_df] = pd.concat([detail_dfs[new_df], new_row], ignore_index=True)
            cols_to_remove.append(col)
    return cols_to_remove

main_dfs = [df if '.' not in df else df.split('.')[0] for df in detail_dfs]
for df in main_dfs:
    print(f"Exploding {df}")
    cols_to_remove = detail_dfs[df].apply(lambda x: explode_df(x, df), axis=1)
    cols_to_remove = [item for sublist in cols_to_remove for item in sublist]
    detail_dfs[df].drop(columns=cols_to_remove, inplace=True)


Exploding Patient
Exploding Encounter
Exploding Condition
Exploding Observation
Exploding Immunization
Exploding CarePlan
Exploding Procedure
Exploding DiagnosticReport
Exploding MedicationRequest
Exploding AllergyIntolerance


In [69]:
for df in detail_dfs:
    print(f"\nResource type: {df}")
    detail_dfs[df].info(show_counts=False, memory_usage=False)


Resource type: Patient
<class 'pandas.core.frame.DataFrame'>
Index: 1054 entries, 0 to 530
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   resourceType          object 
 1   id                    object 
 2   gender                object 
 3   birthDate             object 
 4   multipleBirthBoolean  object 
 5   resourceType          object 
 6   text.status           object 
 7   text.div              object 
 8   deceasedDateTime      object 
 9   multipleBirthInteger  float64
dtypes: float64(1), object(9)
Resource type: Encounter
<class 'pandas.core.frame.DataFrame'>
Index: 9472 entries, 1 to 5101
Data columns (total 8 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   resourceType       object
 1   id                 object
 2   status             object
 3   resourceType       object
 4   class.code         object
 5   patient.reference  object
 6   period.start       object
 7   period.end     

### Patient adjust data

This extension in the patient data is unclear. We will perform data exploration and make necessary changes to it.

In [70]:
df = detail_dfs['Patient.extension']
df['url'].unique()
df['extension_type'] = df['url'].apply(lambda x: x.split('/')[-1])
df.head()

,url,valueCodeableConcept,valueAddress,valueString,Patient_id,valueCode,extension_type
0,http://hl7.org/fhir/StructureDefinition/us-cor...,{'coding': [{'system': 'http://hl7.org/fhir/v3...,NaN,NaN,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,NaN,us-core-race
1,http://hl7.org/fhir/StructureDefinition/us-cor...,{'coding': [{'system': 'http://hl7.org/fhir/v3...,NaN,NaN,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,NaN,us-core-ethnicity
2,http://standardhealthrecord.org/fhir/extension...,NaN,"{'city': 'Lowell', 'state': 'MA'}",NaN,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,NaN,placeOfBirth
3,http://hl7.org/fhir/StructureDefinition/patien...,NaN,NaN,Lukas138 Beer653,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,NaN,patient-mothersMaidenName
4,http://standardhealthrecord.org/fhir/extension...,NaN,NaN,Gerson187 Hayes97,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,NaN,fathersName


In [71]:
for field in ['valueCodeableConcept', 'valueAddress', 'valueCode', 'valueString']:
    print(df[field].value_counts())

valueCodeableConcept
{'coding': [{'system': 'http://hl7.org/fhir/v3/Ethnicity', 'code': '2186-5', 'display': 'Nonhispanic'}], 'text': 'ethnicity'}         480
{'coding': [{'system': 'http://hl7.org/fhir/v3/Race', 'code': '2106-3', 'display': 'White'}], 'text': 'race'}                         412
{'coding': [{'system': 'http://hl7.org/fhir/v3/Race', 'code': '2131-1', 'display': 'Other'}], 'text': 'race'}                          51
{'coding': [{'system': 'http://hl7.org/fhir/v3/Ethnicity', 'code': '2180-8', 'display': 'Puerto_rican'}], 'text': 'ethnicity'}         37
{'coding': [{'system': 'http://hl7.org/fhir/v3/Race', 'code': '2054-5', 'display': 'Black'}], 'text': 'race'}                          35
{'coding': [{'system': 'http://hl7.org/fhir/v3/Race', 'code': '2028-9', 'display': 'Asian'}], 'text': 'race'}                          33
{'coding': [{'system': 'http://hl7.org/fhir/v3/Ethnicity', 'code': '2148-5', 'display': 'Mexican'}], 'text': 'ethnicity'}               9
{'coding': [{

The actual meaning value in 'valueCodeableConcept' is display; \
'valueAddress' could be displayed as string, seperated by comma; \
'valueCode' can also be displayed as string. \
I should extract all of them and store these values into 'value' field as a list. 

In [72]:
def extract_extension(row):
    if pd.notna(row['valueCodeableConcept']):
        if isinstance(row['valueCodeableConcept'], dict):
            row['value'] = "".join([v['display'] for v in row['valueCodeableConcept']['coding']])
    if pd.notna(row['valueAddress']):
        if isinstance(row['valueAddress'], dict):
            row['value'] = ", ".join(row['valueAddress'].values())
    if pd.notna(row['valueCode']):
        row['value'] = row['valueCode']
    if pd.notna(row['valueString']):
        row['value'] = row['valueString']
    return row

df = df.apply(extract_extension, axis=1)
df.drop(columns=['valueCodeableConcept', 'valueAddress', 'valueCode', 'valueString', 'url'], inplace=True)

detail_dfs['Patient.extension'] = df
df.head()

,Patient_id,extension_type,value
0,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,us-core-race,White
1,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,us-core-ethnicity,Nonhispanic
2,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,placeOfBirth,"Lowell, MA"
3,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,patient-mothersMaidenName,Lukas138 Beer653
4,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,fathersName,Gerson187 Hayes97


In [73]:
df = detail_dfs['Patient.address']
df.explode('extension')
def explode_latitud_longitud(row):
    if pd.notna(row['extension']):
        if isinstance(row['extension'], list):
            for ext in row['extension'][0]['extension']:
                if ext['url'] == 'latitude':
                    row['latitude'] = ext['valueDecimal']
                if ext['url'] == 'longitude':
                    row['longitude'] = ext['valueDecimal']
    return row

df = df.apply(explode_latitud_longitud, axis=1)

In [74]:
def extract_first_line(row):
	row['line'] = row['line'][0]
	return row

df = df.apply(extract_first_line, axis=1)
df.drop(columns=['extension'], inplace=True)

detail_dfs['Patient.address'] = df
df.head()

,line,city,state,postalCode,Patient_id,latitude,longitude
0,4711 Powlowski Loop,Shrewsbury,MA,01545,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,42.311666,-71.701575
1,348 Rodolfo Trafficway,Walpole,MA,02081,6ca82ef7-49ec-48c7-bc21-ea7ebdb9afe4,42.157578,-71.202704
2,412 Jacey Spur,Dudley,MA,01571,42c705df-3b17-4f46-a045-911bd542a5d1,42.068571,-71.950993
3,84443 Axel Mountains,Shrewsbury,MA,01546,742632b7-8105-4453-80d7-430a065f8010,42.259827,-71.677556
4,1506 Arvel Route,Burlington,MA,01803,8fd20752-2123-4486-9395-a1e4922920bd,42.508575,-71.192398


In [75]:
df = detail_dfs['Patient.telecom']
df = df.explode('extension')
def explode_telecom(row):
    if pd.notna(row['extension']):
        field = row['extension']['url'].split('/')[-1]
        value = [v for v in row['extension'].keys() if v.startswith('value')][0]
        row[field] = row['extension'][value]
    return row
df = df.apply(explode_telecom, axis=1)

In [76]:
df.drop(columns=['extension'], inplace=True)

detail_dfs['Patient.telecom'] = df
df.head()

,system,value,use,Patient_id,okayToLeaveMessage
0,phone,493.648.2471 x26272,home,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,True
1,phone,954.817.6687,home,6ca82ef7-49ec-48c7-bc21-ea7ebdb9afe4,True
2,phone,133-061-0037 x987,home,42c705df-3b17-4f46-a045-911bd542a5d1,True
3,phone,(494) 819-8705 x4766,home,742632b7-8105-4453-80d7-430a065f8010,True
4,phone,(439) 334-9923 x88504,home,8fd20752-2123-4486-9395-a1e4922920bd,True


In [77]:
df = detail_dfs['Patient.name']
df = df.explode(['given', 'prefix', 'suffix'])

detail_dfs['Patient.name'] = df
df[df['suffix'].notna()]

,use,family,given,prefix,Patient_id,suffix
11,official,Reichel133,Caleb562,Mrs.,086bd6d1-a84c-4fa7-98ae-39b78d39113b,PhD
400,official,Beatty45,Felipe505,Ms.,e6e7c630-73c8-48d3-8f1b-05638b4162a1,JD
486,official,Osinski213,Sabrina736,Mrs.,e86759ad-c212-4a26-8b73-041c829bf5c3,JD


In [78]:
df = detail_dfs['Patient.identifier']

df['type'] = df['system'].apply(lambda x: x.split('/')[-1])
df.drop(columns=['system'], inplace=True)

detail_dfs['Patient.identifier'] = df
df.head()

,value,type,Patient_id
0,0000e4c0-2057-4c43-a90e-33891c7bc097,synthea,80bb41d3-ebb4-42e6-b087-6ee89b0463a9
1,999664433,us-ssn,80bb41d3-ebb4-42e6-b087-6ee89b0463a9
2,S99957033,urn:oid:2.16.840.1.113883.4.3.25,80bb41d3-ebb4-42e6-b087-6ee89b0463a9
3,001be922-fe94-4542-84e9-43623e820da7,synthea,6ca82ef7-49ec-48c7-bc21-ea7ebdb9afe4
4,999921242,us-ssn,6ca82ef7-49ec-48c7-bc21-ea7ebdb9afe4


In [79]:
df = detail_dfs['Patient.maritalStatus.coding']
df.drop(columns=['system'], inplace=True)

detail_dfs['Patient.maritalStatus'] = df

df.head()

,code,Patient_id
0,M,80bb41d3-ebb4-42e6-b087-6ee89b0463a9
1,M,6ca82ef7-49ec-48c7-bc21-ea7ebdb9afe4
2,M,42c705df-3b17-4f46-a045-911bd542a5d1
3,S,742632b7-8105-4453-80d7-430a065f8010
4,M,8fd20752-2123-4486-9395-a1e4922920bd


In [80]:
# Rename 'code' column to 'maritalStatus' in Patient.maritalStatus.coding
detail_dfs['Patient.maritalStatus.coding'].rename(columns={'code': 'maritalStatus'}, inplace=True)

# Merge Patient.maritalStatus.coding with Patient
merged_df = detail_dfs['Patient'].merge(detail_dfs['Patient.maritalStatus.coding'], left_on='id', right_on='Patient_id', how='left')

# Drop the redundant 'Patient_id' column
merged_df.drop(columns=['Patient_id'], inplace=True)

# Update the detail_dfs dictionary with the merged dataframe
detail_dfs['Patient'] = merged_df

# Display the merged dataframe
merged_df.head()

,resourceType,id,gender,birthDate,multipleBirthBoolean,resourceType,text.status,text.div,deceasedDateTime,multipleBirthInteger,maritalStatus
0,Patient,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,female,1959-08-09,False,Patient,generated,"<div>Generated by <a href=""https://github.com/...",NaN,NaN,M
1,Patient,6ca82ef7-49ec-48c7-bc21-ea7ebdb9afe4,female,1936-05-13,False,Patient,generated,"<div>Generated by <a href=""https://github.com/...",NaN,NaN,M
2,Patient,42c705df-3b17-4f46-a045-911bd542a5d1,male,1948-10-05,False,Patient,generated,"<div>Generated by <a href=""https://github.com/...",NaN,NaN,M
3,Patient,742632b7-8105-4453-80d7-430a065f8010,female,1967-03-29,False,Patient,generated,"<div>Generated by <a href=""https://github.com/...",NaN,NaN,S
4,Patient,8fd20752-2123-4486-9395-a1e4922920bd,female,1982-02-10,False,Patient,generated,"<div>Generated by <a href=""https://github.com/...",NaN,NaN,M


In [81]:
df = detail_dfs['Patient']

# Adjust the multipleBirthInteger column
df.loc[df['multipleBirthInteger'].notna() & (df['multipleBirthInteger'] > 0), 'multipleBirthBoolean'] = True

df['birthDate'] = pd.to_datetime(df['birthDate'])
df.drop(columns=['text.status', 'text.div', 'resourceType'], inplace=True)

detail_dfs['Patient'] = df
df.head()

,id,gender,birthDate,multipleBirthBoolean,deceasedDateTime,multipleBirthInteger,maritalStatus
0,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,female,1959-08-09,False,NaN,NaN,M
1,6ca82ef7-49ec-48c7-bc21-ea7ebdb9afe4,female,1936-05-13,False,NaN,NaN,M
2,42c705df-3b17-4f46-a045-911bd542a5d1,male,1948-10-05,False,NaN,NaN,M
3,742632b7-8105-4453-80d7-430a065f8010,female,1967-03-29,False,NaN,NaN,S
4,8fd20752-2123-4486-9395-a1e4922920bd,female,1982-02-10,False,NaN,NaN,M


### Encounter adjust data

In [82]:
df = detail_dfs['Encounter.type']

df = df.explode('coding')

df['type_code'] = df['coding'].apply(lambda x: x['code'])

df.drop(columns=['coding'], inplace=True)

detail_dfs['Encounter.type'] = df
df.head()

,text,Encounter_id,type_code
0,Encounter for 'check-up',50c5c5a1-814d-4a53-8407-129c9048f94a,185349003
1,Encounter for problem,42d937a4-c092-4d0f-87a5-8eec389cedfa,185347001
2,Outpatient Encounter,eaff98cf-528d-4abb-84db-88432aabd27b,185349003
3,Outpatient Encounter,a14ca3d2-03e0-4d41-8d1a-77cba28bced7,185349003
4,Outpatient Encounter,f64fee9b-4eef-4238-a887-0f702c95e039,185349003


In [83]:
df = detail_dfs['Encounter.reason.coding']

df.drop(columns=['system'], inplace=True)
df.rename(columns={'code': 'reason_code', 'display': 'reason_display'}, inplace=True)

detail_dfs['Encounter.reason'] = df
df.head()

,reason_code,reason_display,Encounter_id
0,307731004,Injury of tendon of the rotator cuff of shoulder,42d937a4-c092-4d0f-87a5-8eec389cedfa
1,65363002,Otitis media,c344d56e-df7f-4d97-82f0-82682f364eb3
2,72892002,Normal pregnancy,d8dabaef-d9ff-4570-81ce-54e56248bd67
3,444814009,Viral sinusitis (disorder),45e7a62c-f129-4bb5-8486-97c2b9ac2e72
4,65363002,Otitis media,c411e6aa-9b2f-4ec5-9549-952ef86189df


In [ ]:
df = detail_dfs['Encounter']

df = df[df['patient.reference'].notna()]

df['period_start'] = pd.to_datetime(df['period.start'])
df['period_end'] = pd.to_datetime(df['period.end'])

df['patient_id'] = df['patient.reference'].apply(lambda x: str(x).split(':')[-1])

type_merged_df = df.merge(detail_dfs['Encounter.type'], left_on='id', right_on='Encounter_id', how='left')
reason_merged_df = type_merged_df.merge(detail_dfs['Encounter.reason.coding'], left_on='id', right_on='Encounter_id', how='left')

reason_merged_df.drop(columns=['Encounter_id_x', 'Encounter_id_y', 'reason_display', 'resourceType', 'period.start', 'period.end', 'patient.reference'], inplace=True)

reason_merged_df = reason_merged_df.rename(columns={'class.code': 'class_code'})

detail_dfs['Encounter'] = reason_merged_df
detail_dfs['Encounter'].head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\4224026848.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['period_start'] = pd.to_datetime(df['period.start'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\4224026848.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['period_start'] = pd.to_datetime(df['period.start'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\4224026848.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed 

,id,status,class.code,period_start,period_end,patient_id,text,type_code,reason_code
0,50c5c5a1-814d-4a53-8407-129c9048f94a,finished,ambulatory,2010-12-05 02:33:19-05:00,2010-12-05 03:07:01-05:00,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,Encounter for 'check-up',185349003,NaN
1,42d937a4-c092-4d0f-87a5-8eec389cedfa,finished,ambulatory,2012-03-02 19:39:24-05:00,2012-03-02 19:39:24-05:00,bb357dc7-2f8a-444e-ba57-8291b6f18bf0,Encounter for problem,185347001,307731004
2,eaff98cf-528d-4abb-84db-88432aabd27b,finished,outpatient,2012-05-22 05:23:52-04:00,2012-05-22 06:23:52-04:00,bb357dc7-2f8a-444e-ba57-8291b6f18bf0,Outpatient Encounter,185349003,NaN
3,a14ca3d2-03e0-4d41-8d1a-77cba28bced7,finished,outpatient,2015-01-19 02:00:44-05:00,2015-01-19 03:00:44-05:00,c71d235f-9b58-4106-b744-c554da5e13fa,Outpatient Encounter,185349003,NaN
4,f64fee9b-4eef-4238-a887-0f702c95e039,finished,outpatient,2011-10-15 19:42:43-04:00,2011-10-15 20:42:43-04:00,509474a0-baf3-48f2-a595-67b6e2b34c87,Outpatient Encounter,185349003,NaN


In [85]:
# Drop the Encounter_id column from Encounter.reason and Encounter.type
detail_dfs['Encounter.reason'].drop(columns=['Encounter_id'], inplace=True)
detail_dfs['Encounter.type'].drop(columns=['Encounter_id'], inplace=True)

# Drop duplicates from Encounter.reason and Encounter.type
detail_dfs['Encounter.reason'].drop_duplicates(inplace=True)
detail_dfs['Encounter.type'].drop_duplicates(inplace=True)

In [86]:
detail_dfs.pop('Encounter.reason.coding', None)

,reason_code,reason_display
0,307731004,Injury of tendon of the rotator cuff of shoulder
1,65363002,Otitis media
2,72892002,Normal pregnancy
3,444814009,Viral sinusitis (disorder)
6,43878008,Streptococcal sore throat (disorder)
8,195662009,Acute viral pharyngitis (disorder)
14,10509002,Acute bronchitis (disorder)
16,233678006,Childhood asthma
38,33737001,Fracture of rib
39,62106007,Concussion with no loss of consciousness


### Condition adjust data

In [87]:
df = detail_dfs['Condition.code.coding']

df.drop(columns=['system'], inplace=True)
df.rename(columns={'code': 'condition_code', 'display': 'condition_display'}, inplace=True)

detail_dfs['Condition.code'] = df
df.head()

,condition_code,condition_display,Condition_id
0,15777000,Prediabetes,4eee0d5a-1999-4af9-8c06-21d8447ab278
1,444814009,Viral sinusitis (disorder),c080d5d5-1c0f-4054-bff6-d062035a6e0e
2,58150001,Fracture of clavicle,ea000905-7a51-4f29-bc3a-8c8cc61a38e6
3,10509002,Acute bronchitis (disorder),76d7f42b-358d-417c-8929-c9144255da69
4,10509002,Acute bronchitis (disorder),93cfd203-eb61-4099-9bd3-73b8fbabf1c8


In [88]:
detail_dfs['Condition'].head()

,resourceType,id,clinicalStatus,verificationStatus,onsetDateTime,abatementDateTime,resourceType,subject.reference,context.reference
2,Condition,4eee0d5a-1999-4af9-8c06-21d8447ab278,active,confirmed,1982-12-31T01:10:52-05:00,NaN,Condition,urn:uuid:bb357dc7-2f8a-444e-ba57-8291b6f18bf0,urn:uuid:aa76d5b2-8587-4dae-b5f9-487ac3a89308
68,Condition,c080d5d5-1c0f-4054-bff6-d062035a6e0e,active,confirmed,2010-07-21T23:56:22-04:00,2010-08-11T02:28:57-04:00,Condition,urn:uuid:bd0acd69-580e-4896-9974-2ad9d041ca0b,urn:uuid:65a721f6-f29c-4d45-8b81-c8a32aef22c1
69,Condition,ea000905-7a51-4f29-bc3a-8c8cc61a38e6,active,confirmed,2010-10-30T03:53:48-04:00,2011-01-15T17:27:00-05:00,Condition,urn:uuid:bd0acd69-580e-4896-9974-2ad9d041ca0b,urn:uuid:2de8efc2-b0a1-414e-acee-2202e0e617ba
112,Condition,76d7f42b-358d-417c-8929-c9144255da69,active,confirmed,2011-12-04T11:01:27-05:00,2011-12-15T14:06:06-05:00,Condition,urn:uuid:8b47143f-2d3b-4425-a872-569af4ec0818,urn:uuid:333ebab1-3eef-43bb-b49c-d5ec064353ab
114,Condition,93cfd203-eb61-4099-9bd3-73b8fbabf1c8,active,confirmed,1982-08-18T07:07:52-04:00,1982-08-26T02:57:15-04:00,Condition,urn:uuid:9bf6df85-12db-49d1-b754-b5adcdbb6115,urn:uuid:e474dde0-32b2-404c-8622-bc008af313e4


In [89]:
df = detail_dfs['Condition']

df['onsetDateTime'] = pd.to_datetime(df['onsetDateTime'])
df['abatementDateTime'] = pd.to_datetime(df['abatementDateTime'])

df = df[df['subject.reference'].notna() & df['context.reference'].notna()]

df['patient_id'] = df['subject.reference'].apply(lambda x: str(x).split(':')[-1])
df['encounter_id'] = df['context.reference'].apply(lambda x: str(x).split(':')[-1])

merged_df = df.merge(detail_dfs['Condition.code'], left_on='id', right_on='Condition_id', how='left')

merged_df.drop(columns=['resourceType', 'subject.reference', 'context.reference', 'Condition_id', 'condition_display'], inplace=True)

detail_dfs['Condition'] = merged_df
detail_dfs['Condition'].head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\2041017584.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['onsetDateTime'] = pd.to_datetime(df['onsetDateTime'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\2041017584.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['abatementDateTime'] = pd.to_datetime(df['abatementDateTime'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\2041017584.py:8: SettingWithCopyWarning: 
A valu

,id,clinicalStatus,verificationStatus,onsetDateTime,abatementDateTime,patient_id,encounter_id,condition_code
0,4eee0d5a-1999-4af9-8c06-21d8447ab278,active,confirmed,1982-12-31 01:10:52-05:00,NaT,bb357dc7-2f8a-444e-ba57-8291b6f18bf0,aa76d5b2-8587-4dae-b5f9-487ac3a89308,15777000
1,c080d5d5-1c0f-4054-bff6-d062035a6e0e,active,confirmed,2010-07-21 23:56:22-04:00,2010-08-11 02:28:57-04:00,bd0acd69-580e-4896-9974-2ad9d041ca0b,65a721f6-f29c-4d45-8b81-c8a32aef22c1,444814009
2,ea000905-7a51-4f29-bc3a-8c8cc61a38e6,active,confirmed,2010-10-30 03:53:48-04:00,2011-01-15 17:27:00-05:00,bd0acd69-580e-4896-9974-2ad9d041ca0b,2de8efc2-b0a1-414e-acee-2202e0e617ba,58150001
3,76d7f42b-358d-417c-8929-c9144255da69,active,confirmed,2011-12-04 11:01:27-05:00,2011-12-15 14:06:06-05:00,8b47143f-2d3b-4425-a872-569af4ec0818,333ebab1-3eef-43bb-b49c-d5ec064353ab,10509002
4,93cfd203-eb61-4099-9bd3-73b8fbabf1c8,active,confirmed,1982-08-18 07:07:52-04:00,1982-08-26 02:57:15-04:00,9bf6df85-12db-49d1-b754-b5adcdbb6115,e474dde0-32b2-404c-8622-bc008af313e4,10509002


In [90]:
detail_dfs['Condition.code'].drop(columns=['Condition_id'], inplace=True)
detail_dfs['Condition.code'].drop_duplicates(inplace=True)
detail_dfs['Condition.code'].value_counts()

condition_code   condition_display                                                               
10509002         Acute bronchitis (disorder)                                                         1
124171000119105  Chronic intractable migraine without aura                                           1
127013003        Diabetic renal disease (disorder)                                                   1
1501000119109    Proliferative diabetic retinopathy due to type II diabetes mellitus (disorder)      1
1551000119108    Nonproliferative diabetic retinopathy due to type 2 diabetes mellitus (disorder)    1
                                                                                                    ..
90560007         Gout                                                                                1
90781000119102   Microalbuminuria due to type 2 diabetes mellitus (disorder)                         1
93761005         Primary malignant neoplasm of colon                          

In [91]:
detail_dfs.pop('Condition.code.coding', None)

,condition_code,condition_display
0,15777000,Prediabetes
1,444814009,Viral sinusitis (disorder)
2,58150001,Fracture of clavicle
3,10509002,Acute bronchitis (disorder)
6,38341003,Hypertension
...,...,...
1404,403191005,Second degree burn
1484,97331000119101,Macular edema and retinopathy due to type 2 di...
1493,196416002,Impacted molars
1585,444470001,Injury of anterior cruciate ligament


### Observation adjust data

In [92]:
df = detail_dfs['Observation.code.coding']

df.drop(columns=['system'], inplace=True)
df.rename(columns={'code': 'observation_code', 'display': 'observation_display'}, inplace=True)

detail_dfs['Observation.code'] = df
df.head()

,observation_code,observation_display,Observation_id
0,55284-4,Blood Pressure,7b141a9c-6c37-4d06-8eb8-4c61c5ca4bf0
1,8302-2,Body Height,cac21874-494f-49f6-8074-1d517a62a7d3
2,29463-7,Body Weight,61d46e20-1a26-4fb9-ba74-e21c01bce443
3,39156-5,Body Mass Index,62b7938f-5fab-4067-8e6a-fc4f8df284d3
4,55284-4,Blood Pressure,2e64f693-fe4f-4405-a0b4-25bd031f0660


In [93]:
df = detail_dfs['Observation.valueCodeableConcept.coding']

df.drop(columns=['system'], inplace=True)
df.rename(columns={'code': 'value_code', 'display': 'value_display'}, inplace=True)

detail_dfs['Observation.valueCodeableConcept'] = df
df.head()

,value_code,value_display,Observation_id
0,87433001,Pulmonary emphysema (disorder),018132b0-b4c9-4b67-84a7-90faa6c1931d
1,87433001,Pulmonary emphysema (disorder),4a4a7a54-396a-442f-9d84-87ffa2fac09c
2,55680006,Drug overdose,fb543c46-051a-4353-8b36-f909a94bdd77
3,185086009,Chronic obstructive bronchitis (disorder),14b749eb-2a71-4526-9277-532892ea1a91
4,87433001,Pulmonary emphysema (disorder),11f34a5d-be7e-45bb-b8df-e8d8cacbafca


In [94]:
df = detail_dfs['Observation.component']

df['code'] = df['code'].apply(lambda x: x['coding'])
df = df.explode('code', ignore_index=True)

# flatten the code column
df.rename(columns={'code': 'component_code'}, inplace=True)
df = pd.concat([pd.json_normalize(df['component_code']), df], axis=1)
df.rename(columns={'code': 'display_code'}, inplace=True)

# flatten the valueQuantity column
df = pd.concat([pd.json_normalize(df['valueQuantity']), df], axis=1)

df.drop(columns=['component_code', 'valueQuantity', 'system', 'code'], inplace=True)
detail_dfs['Observation.component'] = df
df.head()

,value,unit,display_code,display,Observation_id
0,136,mmHg,8480-6,Systolic Blood Pressure,7b141a9c-6c37-4d06-8eb8-4c61c5ca4bf0
1,86,mmHg,8462-4,Diastolic Blood Pressure,7b141a9c-6c37-4d06-8eb8-4c61c5ca4bf0
2,133,mmHg,8480-6,Systolic Blood Pressure,2e64f693-fe4f-4405-a0b4-25bd031f0660
3,72,mmHg,8462-4,Diastolic Blood Pressure,2e64f693-fe4f-4405-a0b4-25bd031f0660
4,109,mmHg,8480-6,Systolic Blood Pressure,2e3a9af1-035a-44a5-97a5-5669a5c02e8b


In [95]:
df = detail_dfs['Observation']

# Rename and retype columns
df['effectiveDateTime'] = pd.to_datetime(df['effectiveDateTime'])
df.rename(columns={'subject.reference': 'patient_id', 'encounter.reference': 'encounter_id'}, inplace=True)
df['patient_id'] = df['patient_id'].apply(lambda x: str(x).split(':')[-1])
df['encounter_id'] = df['encounter_id'].apply(lambda x: str(x).split(':')[-1])

# Merge with Observation.valueCodeableConcept
df = df.merge(detail_dfs['Observation.valueCodeableConcept'], left_on='id', right_on='Observation_id', how='left')
df.drop(columns=['Observation_id', 'value_display'], inplace=True)

# Merge with Observation.code
df = df.merge(detail_dfs['Observation.code'], left_on='id', right_on='Observation_id', how='left')
df.drop(columns=['Observation_id', 'observation_display'], inplace=True)

detail_dfs['Observation.result'] = df[['id', 'valueQuantity.value', 'valueQuantity.unit', 'valueQuantity.system', 'valueQuantity.code', 'value_code', 'observation_code']]

df.drop(columns=['valueQuantity.value', 'valueQuantity.unit', 'valueQuantity.system', 'valueQuantity.code', 'value_code', 'resourceType'], inplace=True)

detail_dfs['Observation'] = df

detail_dfs['Observation'].head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\2802718056.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['effectiveDateTime'] = pd.to_datetime(df['effectiveDateTime'])


,id,status,effectiveDateTime,patient_id,encounter_id,observation_code
0,7b141a9c-6c37-4d06-8eb8-4c61c5ca4bf0,final,2010-05-12 10:33:51-04:00,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,2c722ad1-5a59-4f6a-90cc-c55f86bbd9c6,55284-4
1,cac21874-494f-49f6-8074-1d517a62a7d3,final,2011-07-02 14:56:51-04:00,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,f5b01614-f569-4e9e-b482-cc0231a7c20a,8302-2
2,61d46e20-1a26-4fb9-ba74-e21c01bce443,final,2011-07-02 14:56:51-04:00,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,f5b01614-f569-4e9e-b482-cc0231a7c20a,29463-7
3,62b7938f-5fab-4067-8e6a-fc4f8df284d3,final,2011-07-02 14:56:51-04:00,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,f5b01614-f569-4e9e-b482-cc0231a7c20a,39156-5
4,2e64f693-fe4f-4405-a0b4-25bd031f0660,final,2012-06-03 09:20:31-04:00,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,1460d856-b01a-4c3b-b8c7-07fd86cac5fe,55284-4


In [96]:
df = detail_dfs['Observation.result']
df.rename(columns={'id': 'Observation_id'}, inplace=True)

df = df.merge(detail_dfs['Observation.component'], left_on='Observation_id', right_on='Observation_id', how='outer')

df['value'] = df['value'].combine_first(df['valueQuantity.value'])
df['unit'] = df['unit'].combine_first(df['valueQuantity.unit'])
df['display_code'] = df['display_code'].combine_first(df['observation_code'])
df['value'] = df['value_code'].combine_first(df['value'])
df.loc[df['value_code'].notna(), 'unit'] = 'conceptable'

df.drop(columns=['valueQuantity.value', 'valueQuantity.unit', 'valueQuantity.system', 'valueQuantity.code', 'value_code', 'observation_code', 'display'], inplace=True)
detail_dfs['Observation.result'] = df
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\2910551018.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'id': 'Observation_id'}, inplace=True)


,Observation_id,value,unit,display_code
0,000846ce-2c8a-4f15-b78d-2a3946dd7511,33.455703,kg/m2,39156-5
1,00094faa-4c6d-408a-b4b2-9cf695b69993,126.733262,kg,29463-7
2,00107bdb-af53-45a2-acb6-692be5af03f1,48.0,%,19926-5
3,0012e3b4-1229-480b-9da4-a131eceeb7e2,141.0,mg/dL,2571-8
4,0014b791-810f-4bae-a534-613f3b20aa89,158.88273,cm,8302-2


In [97]:
df = detail_dfs['Observation.component']

df = df[['display_code', 'display']]
df.drop_duplicates(inplace=True)

df.rename(columns={'display': 'observation_display', 'display_code': 'observation_code'}, inplace=True)

detail_dfs['Observation.code'] = pd.concat([detail_dfs['Observation.code'], df], ignore_index=True)
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\788051776.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\788051776.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'display': 'observation_display', 'display_code': 'observation_code'}, inplace=True)


,observation_code,observation_display
0,8480-6,Systolic Blood Pressure
1,8462-4,Diastolic Blood Pressure


In [98]:
detail_dfs['Observation.code'].drop(columns=['Observation_id'], inplace=True)
detail_dfs['Observation.code'].drop_duplicates(inplace=True)

detail_dfs['Observation.valueCodeableConcept'].drop(columns=['Observation_id'], inplace=True)
detail_dfs['Observation.valueCodeableConcept'].drop_duplicates(inplace=True)

detail_dfs.pop('Observation.valueCodeableConcept.coding', None)
detail_dfs.pop('Observation.code.coding', None)

detail_dfs['Observation.code'].info()
detail_dfs['Observation.valueCodeableConcept'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 0 to 19380
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   observation_code     52 non-null     object
 1   observation_display  52 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 14
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   value_code     6 non-null      object
 1   value_display  6 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


### DiagnosticResult adjust data

In [ ]:
df = detail_dfs['DiagnosticReport.code.coding']
df.drop(columns=['system'], inplace=True)

detail_dfs['DiagnosticReport.code'] = df
df.head()


,code,display,DiagnosticReport_id
0,57698-3,Lipid Panel,ff1b01ef-8d21-48da-8f7e-b3db4d62417c
1,51990-0,Basic Metabolic Panel,60a3adfa-b06c-44bd-9285-c181b8c3d88e
2,57698-3,Lipid Panel,d50e1609-872b-43e7-8679-42a980260ac3
3,51990-0,Basic Metabolic Panel,91659759-ce79-431f-bc24-fcffa0b0f53d
4,57698-3,Lipid Panel,b831beff-cda7-42f8-b524-24f3512766ed


In [146]:
detail_dfs['DiagnosticReport.code'].head()

,code,display
0,57698-3,Lipid Panel
1,51990-0,Basic Metabolic Panel
7,69409-1,U.S. standard certificate of death - 2003 revi...


In [100]:
df = detail_dfs['DiagnosticReport.performer']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189 entries, 0 to 1188
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   display              1189 non-null   object
 1   DiagnosticReport_id  1189 non-null   object
dtypes: object(2)
memory usage: 18.7+ KB


In [101]:
df = detail_dfs['DiagnosticReport.result']
df['reference'] = df['reference'].apply(lambda x: str(x).split(':')[-1])

detail_dfs['DiagnosticReport.result'] = df
df.head()

,reference,display,DiagnosticReport_id
0,98ab19e8-4623-4e30-8782-83620ed34f17,Total Cholesterol,ff1b01ef-8d21-48da-8f7e-b3db4d62417c
1,aeff25de-10b9-4a2b-8740-8aa66d34f1cd,Triglycerides,ff1b01ef-8d21-48da-8f7e-b3db4d62417c
2,7ae8d33d-97a2-4d62-b645-f88654478e67,Low Density Lipoprotein Cholesterol,ff1b01ef-8d21-48da-8f7e-b3db4d62417c
3,3a82f21d-f33d-497f-99fb-523b8c7154af,High Density Lipoprotein Cholesterol,ff1b01ef-8d21-48da-8f7e-b3db4d62417c
4,d1137901-93eb-4965-bf0d-7a6a62f8e138,Glucose,60a3adfa-b06c-44bd-9285-c181b8c3d88e


In [149]:
detail_dfs['DiagnosticReport.result']['display'].value_counts()

display
Glucose                                              619
Sodium                                               619
Calcium                                              619
Creatinine                                           619
Urea Nitrogen                                        619
Potassium                                            619
Chloride                                             619
Carbon Dioxide                                       619
Total Cholesterol                                    532
High Density Lipoprotein Cholesterol                 532
Low Density Lipoprotein Cholesterol                  532
Triglycerides                                        532
Cause of Death [US Standard Certificate of Death]     38
Name: count, dtype: Int64

In [ ]:
df = detail_dfs['DiagnosticReport']

df['effectiveDateTime'] = pd.to_datetime(df['effectiveDateTime'])
df['issued'] = pd.to_datetime(df['issued'])

df['subject_id'] = df['subject.reference'].apply(lambda x: str(x).split(':')[-1])
df['encounter_id'] = df['encounter.reference'].apply(lambda x: str(x).split(':')[-1])

df = df.merge(detail_dfs['DiagnosticReport.code'], left_on='id', right_on='DiagnosticReport_id', how='left')
df.drop(columns=['DiagnosticReport_id', 'display'], inplace=True)
df = df.rename(columns={'code': 'report_code'})

detail_dfs['DiagnosticReport.code'].drop(columns=['DiagnosticReport_id'], inplace=True)
detail_dfs['DiagnosticReport.code'].drop_duplicates(inplace=True)

df.drop(columns=['subject.reference', 'encounter.reference', 'resourceType'], inplace=True)

detail_dfs['DiagnosticReport'] = df
df.head()

,id,status,effectiveDateTime,issued,subject_id,encounter_id,report_code
0,ff1b01ef-8d21-48da-8f7e-b3db4d62417c,final,2017-02-22 13:52:56-05:00,2017-02-22 13:52:56-05:00,117335bb-33ff-4897-ac27-af4b3f11135f,5948265c-890d-4b41-9b89-61b195cfa4c5,57698-3
1,60a3adfa-b06c-44bd-9285-c181b8c3d88e,final,2011-06-02 20:58:47-04:00,2011-06-02 20:58:47-04:00,3a5357d9-13f3-42de-8fdc-14a1a6cae148,d53f9c2a-250a-49a9-9f8d-321ca30034bb,51990-0
2,d50e1609-872b-43e7-8679-42a980260ac3,final,2015-05-15 04:15:28-04:00,2015-05-15 04:15:28-04:00,aa3973d9-b64f-4a36-8cb6-f2719080b52f,638fee81-9b8f-4896-8536-407e34db67c0,57698-3
3,91659759-ce79-431f-bc24-fcffa0b0f53d,final,2010-07-28 22:24:43-04:00,2010-07-28 22:24:43-04:00,24950485-2ecd-4405-bb76-e92a3259adc4,3bf22b8b-0a73-4bce-a6d9-670c1ff5c4c6,51990-0
4,b831beff-cda7-42f8-b524-24f3512766ed,final,2010-07-05 17:41:42-04:00,2010-07-05 17:41:42-04:00,edfbc3f7-9ec5-472c-bd5c-ac9041927441,f2118807-b201-48e3-8271-8074781be72c,57698-3


### AllergyIntolerance adjust data

In [ ]:
df = detail_dfs['AllergyIntolerance']

df.drop(columns=['resourceType'], inplace=True)
df = df.dropna(how='all')
df = df.explode('code.coding', ignore_index=True)
df['display'] = df['code.coding'].apply(lambda x: x['display'])
df['code'] = df['code.coding'].apply(lambda x: x['code'])

df['patient_id'] = df['patient.reference'].apply(lambda x: str(x).split(':')[-1])
df['assertedDate'] = pd.to_datetime(df['assertedDate'])

df['category'] = df['category'].apply(lambda x: ','.join(x))

df.drop(columns=['code.coding', 'patient.reference'], inplace=True)

detail_dfs['AllergyIntolerance'] = df
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11140\3571892836.py:10: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['assertedDate'] = pd.to_datetime(df['assertedDate'])


,clinicalStatus,type,category,criticality,assertedDate,display,code,patient_id
0,active,allergy,food,low,1945-01-03 16:27:18-04:00,Allergy to fish,417532002,a0dcd747-1d40-40fd-8e42-1f9395f034ca
1,active,allergy,food,high,1945-01-03 16:27:18-04:00,Allergy to grass pollen,418689008,a0dcd747-1d40-40fd-8e42-1f9395f034ca
2,active,allergy,food,low,1945-01-03 16:27:18-04:00,Allergy to mould,419474003,a0dcd747-1d40-40fd-8e42-1f9395f034ca
3,active,allergy,food,high,1989-08-30 10:46:39-04:00,Shellfish allergy,300913006,ddd6fcc5-4a60-450d-a55c-927113ec312e
4,active,allergy,food,low,1989-08-30 10:46:39-04:00,Allergy to tree pollen,419263009,ddd6fcc5-4a60-450d-a55c-927113ec312e


### CarePlan adjust data

In [124]:
def explode_flatten_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function explodes lists and flattens dictionaries in a dataframe until there are none.
    """
    while True:
        # Identify columns that need to be exploded or flattened
        list_cols = [col for col in df.columns if isinstance(df[df[col].notna()][col].iloc[0], list)]
        dict_cols = [col for col in df.columns if isinstance(df[df[col].notna()][col].iloc[0], dict)]
        
        if not list_cols and not dict_cols:
            break
        
        # Explode list columns
        for col in list_cols:
            df = df.explode(col, ignore_index=True)
        
        # Flatten dictionary columns
        for col in dict_cols:
            normalized_data = pd.json_normalize(df[col])
            normalized_data.columns = [f"{col}.{subcol}" for subcol in normalized_data.columns]
            df = pd.concat([df.drop(columns=[col]), normalized_data], axis=1)
    
    return df


In [125]:
from typing import List


def extract_id(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """
    Extract the id from a reference column
    """
    for col in cols:
        new_col = col.split('.')[0] + '_id'
        df[new_col] = df[col].apply(lambda x: str(x).split(':')[-1])
        df.loc[df[col].isna(), new_col] = np.nan
        df.drop(columns=[col], inplace=True)
    return df

In [126]:
df = detail_dfs['CarePlan']

df.drop(columns=['resourceType'], inplace=True)

df = explode_flatten_df(df)

df = extract_id(df, ['subject.reference', 'context.reference', 'addresses.reference'])

df.drop(columns=['category.coding.system', 'activity.detail.code.coding.system'], inplace=True)

detail_dfs['CarePlan.category'] = df[['category.coding.code', 'category.coding.display']].drop_duplicates()
detail_dfs['CarePlan.activity'] = df[['activity.detail.code.coding.code', 'activity.detail.code.coding.display']].drop_duplicates()

df.rename(columns={'category.coding.code': 'category_code', 'activity.detail.code.coding.code': 'activity_code', 'activity.detail.status': 
'activity_status'}, inplace=True)
df.drop(columns=['category.coding.display', 'activity.detail.code.coding.display'], inplace=True)

detail_dfs['CarePlan'] = df
df.head()

,status,period.start,period.end,activity_status,category_code,activity_code,subject_id,context_id,addresses_id
0,active,1968-03-07,NaN,in-progress,698360004,160670007,c71d235f-9b58-4106-b744-c554da5e13fa,0295554d-c9f6-402f-bcbc-0a818521f4c5,66868a39-d41a-4acb-a3a2-202093909119
1,active,1968-03-07,NaN,in-progress,698360004,229065009,c71d235f-9b58-4106-b744-c554da5e13fa,0295554d-c9f6-402f-bcbc-0a818521f4c5,66868a39-d41a-4acb-a3a2-202093909119
2,completed,1996-03-14,1996-05-12,completed,53950000,304510005,509474a0-baf3-48f2-a595-67b6e2b34c87,d9db693d-4bba-4571-bba9-ae6c1575b945,NaN
3,completed,1996-03-14,1996-05-12,completed,53950000,371605008,509474a0-baf3-48f2-a595-67b6e2b34c87,d9db693d-4bba-4571-bba9-ae6c1575b945,NaN
4,completed,2001-12-13,2002-06-25,completed,53950000,304510005,509474a0-baf3-48f2-a595-67b6e2b34c87,d9db693d-4bba-4571-bba9-ae6c1575b945,NaN


### Immunization adjust data

In [127]:
df = detail_dfs['Immunization']

df.drop(columns=['resourceType'], inplace=True)

df = explode_flatten_df(df)

df = extract_id(df, ['patient.reference', 'encounter.reference'])

df.drop(columns=['vaccineCode.coding.system'], inplace=True)

detail_dfs['Immunization.vaccineCode'] = df[['vaccineCode.coding.code', 'vaccineCode.coding.display']].drop_duplicates()

df.rename(columns={'vaccineCode.coding.code': 'vaccine_code'}, inplace=True)
df.drop(columns=['vaccineCode.coding.display'], inplace=True)

detail_dfs['Immunization'] = df
df.head()

,status,date,wasNotGiven,primarySource,vaccine_code,patient_id,encounter_id
0,completed,2016-05-13T09:03:20-04:00,False,True,140,80bb41d3-ebb4-42e6-b087-6ee89b0463a9,c4119ed9-02d5-4fb3-a51e-7d7cccdf4085
1,completed,2010-09-28T21:56:11-04:00,False,True,140,509474a0-baf3-48f2-a595-67b6e2b34c87,d9db693d-4bba-4571-bba9-ae6c1575b945
2,completed,2011-10-15T19:42:43-04:00,False,True,140,509474a0-baf3-48f2-a595-67b6e2b34c87,f64fee9b-4eef-4238-a887-0f702c95e039
3,completed,2013-05-10T03:21:09-04:00,False,True,49,65dfe191-c7ef-4148-8f12-49f832525d4c,27bcacf8-f83d-422c-8c1e-ea99b8ed8c66
4,completed,2013-07-19T22:54:37-04:00,False,True,10,65dfe191-c7ef-4148-8f12-49f832525d4c,5678920f-d282-46dc-be87-2a0a93563f15


### MedicationRequest adjust data

In [ ]:
df = detail_dfs['MedicationRequest']

df.drop(columns=['resourceType'], inplace=True)
df = explode_flatten_df(df)

df = extract_id(df, ['patient.reference', 'context.reference', 'reasonReference.reference'])

df['id'] = df['dateWritten'] + "_" + df['patient_id']

def classify_medication(row):
    if pd.isna(row['dosageInstruction.asNeededBoolean']):
        return 'no instructions'
    elif row['dosageInstruction.asNeededBoolean']:
        return 'as needed'
    else:
        return 'scheduled'
    
df['medication_classification'] = df.apply(classify_medication, axis=1)

# DimMedication
# Extract unique medication information with relevant columns for medication coding and display.
detail_dfs['MedicationRequest.medication'] = df[['id', 'medicationCodeableConcept.coding.code', 'medicationCodeableConcept.coding.display']
                                                 ].drop_duplicates().rename(
    columns={'id': 'MedicationRequest_id',
             'medicationCodeableConcept.coding.code': 'CodingCode', 
             'medicationCodeableConcept.coding.display': 'CodingDisplay'}
)

# DimDosageInstruction
# Capture dosage instructions, including sequence, frequency, and period.
detail_dfs['MedicationRequest.dosageInstruction'] = df[['id', 'dosageInstruction.sequence', 
                                                        'dosageInstruction.asNeededBoolean', 
                                                        'dosageInstruction.timing.repeat.frequency',  
                                                        'dosageInstruction.timing.repeat.period', 
                                                        'dosageInstruction.timing.repeat.periodUnit', 
                                                        'dosageInstruction.doseQuantity.value',
                                                        'dosageInstruction.additionalInstructions.coding.code']].drop_duplicates().rename(
    columns={'id': 'MedicationRequest_id',
             'dosageInstruction.sequence': 'Sequence',
             'dosageInstruction.asNeededBoolean': 'AsNeededBoolean',
             'dosageInstruction.timing.repeat.frequency': 'Frequency',
             'dosageInstruction.timing.repeat.period': 'Period',
             'dosageInstruction.timing.repeat.periodUnit': 'PeriodUnit',
             'dosageInstruction.doseQuantity.value': 'DoseQuantityValue',
             'dosageInstruction.additionalInstructions.coding.code': 'AdditionalInstructionCode'}
)
detail_dfs['MedicationRequest.dosageInstruction'].dropna(subset=['Sequence'], inplace=True)

# DimAdditionalInstruction
# Extract additional instructions for dosage with coding system, code, and display information.
detail_dfs['MedicationRequest.additionalInstruction'] = df[['dosageInstruction.additionalInstructions.coding.code', 
                                                            'dosageInstruction.additionalInstructions.coding.display']].drop_duplicates().rename(
    columns={'dosageInstruction.additionalInstructions.coding.code': 'InstructionCode',
             'dosageInstruction.additionalInstructions.coding.display': 'InstructionDisplay'}
)
detail_dfs['MedicationRequest.additionalInstruction'].dropna(subset=['InstructionCode'], inplace=True)

# DimDispenseRequest
# Capture dispense request details with expected supply duration and quantity information.
detail_dfs['MedicationRequest.dispenseRequest'] = df[['id', 
                                                      'dispenseRequest.numberOfRepeatsAllowed', 
                                                      'dispenseRequest.quantity.value', 
                                                      'dispenseRequest.quantity.unit', 
                                                      'dispenseRequest.expectedSupplyDuration.value', 
                                                      'dispenseRequest.expectedSupplyDuration.unit', 
                                                      'dispenseRequest.expectedSupplyDuration.code']].drop_duplicates().rename(
    columns={'id': 'MedicationRequest_id',
             'dispenseRequest.numberOfRepeatsAllowed': 'NumberOfRepeatsAllowed',
             'dispenseRequest.quantity.value': 'QuantityValue', 
             'dispenseRequest.quantity.unit': 'QuantityUnit',
             'dispenseRequest.expectedSupplyDuration.value': 'ExpectedSupplyDurationValue', 
             'dispenseRequest.expectedSupplyDuration.unit': 'ExpectedSupplyDurationUnit',
             'dispenseRequest.expectedSupplyDuration.code': 'ExpectedSupplyDurationCode'}
)
detail_dfs['MedicationRequest.dispenseRequest'].dropna(subset=['QuantityValue'], inplace=True)

# Main Medication Request Table
# Retain only high-level columns, including medication code and metadata for main medication request table.
main_df = df[['id', 'status', 'dateWritten', 'patient_id', 'context_id', 'reasonReference_id', 
              'medicationCodeableConcept.coding.code', 'medication_classification']].drop_duplicates().rename(
    columns={'id': 'MedicationRequest_id',
             'medicationCodeableConcept.coding.code': 'MedicationCode'}
)

detail_dfs['MedicationRequest'] = main_df
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2544 entries, 0 to 2543
Data columns (total 29 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   status                                                   1291 non-null   object 
 1   dateWritten                                              1291 non-null   object 
 2   dispenseRequest.numberOfRepeatsAllowed                   90 non-null     float64
 3   dispenseRequest.quantity.value                           90 non-null     float64
 4   dispenseRequest.quantity.unit                            90 non-null     object 
 5   dispenseRequest.expectedSupplyDuration.value             90 non-null     float64
 6   dispenseRequest.expectedSupplyDuration.unit              90 non-null     object 
 7   dispenseRequest.expectedSupplyDuration.system            90 non-null     object 
 8   dispenseRequest.expectedSupp

In [150]:
detail_dfs['MedicationRequest.additionalInstruction'].head()

,MedicationRequest_id,InstructionCode,InstructionDisplay
10,2015-01-22_98be5239-0275-45df-96b0-4faa7995fad9,418577003,Take at regular intervals. Complete the prescr...
21,2010-03-24_3877e505-ecc0-4a70-8e58-9ac93fb301a5,418577003,Take at regular intervals. Complete the prescr...
22,2013-02-02_fa501926-e785-4761-bec5-bc53de5f4222,418577003,Take at regular intervals. Complete the prescr...
30,2010-05-23_2e1e5ade-594b-4ef9-9313-4dac10ac1881,418577003,Take at regular intervals. Complete the prescr...
32,2013-07-26_aa84af31-90c0-4274-b207-3e7d1b8e6953,418577003,Take at regular intervals. Complete the prescr...


### Procedure adjust data

In [129]:
df = detail_dfs['Procedure']

df.drop(columns=['resourceType'], inplace=True)
df = explode_flatten_df(df)
df.dropna(how='all', inplace=True)
df['performedPeriod.start'] = df['performedPeriod.start'].combine_first(df['performedDateTime'])
df['performedPeriod.end'] = df['performedPeriod.end'].combine_first(df['performedDateTime'])
df.drop(columns=['performedDateTime'], inplace=True)

detail_dfs['Procedure.type'] = df[['code.coding.code', 'code.coding.display']].drop_duplicates().rename(
    columns={'code.coding.code': 'ProcedureCode', 'code.coding.display': 'ProcedureDisplay'}
)

df = extract_id(df, ['subject.reference', 'encounter.reference', 'reasonReference.reference'])

df.drop(columns=['code.coding.system', 'code.text', 'code.coding.display'], inplace=True)
df = df.rename(columns={'code.coding.code': 'ProcedureCode', 'code.coding.display': 'ProcedureDisplay'})

detail_dfs['Procedure'] = df
df.head()

,status,performedPeriod.start,performedPeriod.end,ProcedureCode,subject_id,encounter_id,reasonReference_id
0,completed,2011-10-15T19:42:43-04:00,2011-10-15T19:42:43-04:00,428191000124101,509474a0-baf3-48f2-a595-67b6e2b34c87,f64fee9b-4eef-4238-a887-0f702c95e039,NaN
1,completed,2013-12-18T11:54:32-05:00,2013-12-18T11:54:32-05:00,428191000124101,1694897b-912e-401f-a135-aa59857a401e,56155626-b08a-44a6-82e8-9b3e862965e5,NaN
2,completed,2014-02-12T21:29:48-05:00,2014-02-12T21:29:48-05:00,274031008,a0dcd747-1d40-40fd-8e42-1f9395f034ca,fc59c225-917b-4485-a399-38c2c1d545a2,5b8bdf82-c4f8-4b8a-ac6b-441ac775cd1d
3,completed,2013-04-21T02:07:56-04:00,2013-04-21T03:08:21-04:00,305428000,00d869e2-6793-4ebf-9340-38bf18d223c4,8625df2e-3c5d-4dc2-83c7-714a126d407e,3933d233-e42b-446f-b6d5-2124f7d650e6
4,completed,2011-11-21T04:02:10-05:00,2011-11-21T04:31:11-05:00,288086009,bd0acd69-580e-4896-9974-2ad9d041ca0b,afee7487-a8bb-48b2-b1bd-774bebdaed00,88c1ae16-3abd-45a8-946e-2dd82565b11f


In [130]:
# Check if code.coding.display is the same as code.text
detail_dfs['Procedure.type'].value_counts().head()

ProcedureCode     ProcedureDisplay                 
1015401000000102  Fecal occult blood test              1
112790001         Nasal sinus endoscopy (procedure)    1
11466000          Cesarean section                     1
117015009         Throat culture (procedure)           1
127783003         Spirometry (procedure)               1
Name: count, dtype: int64

### Overview
This part will show what dataframe we have so far


In [147]:
dfs = [df for df in detail_dfs]
dfs.sort()
print(*dfs, sep='\n')

AllergyIntolerance
CarePlan
CarePlan.activity
CarePlan.category
Condition
Condition.code
DiagnosticReport
DiagnosticReport.code
DiagnosticReport.performer
DiagnosticReport.result
Encounter
Encounter.reason
Encounter.type
Immunization
Immunization.vaccineCode
MedicationRequest
MedicationRequest.additionalInstruction
MedicationRequest.dispenseRequest
MedicationRequest.dosageInstruction
MedicationRequest.medication
Observation
Observation.code
Observation.component
Observation.result
Observation.valueCodeableConcept
Patient
Patient.address
Patient.extension
Patient.identifier
Patient.maritalStatus
Patient.name
Patient.photo
Patient.telecom
Procedure
Procedure.type


In [148]:
# Remove .coding from the dfs
detail_dfs.pop('DiagnosticReport.code.coding', None)
detail_dfs.pop('Patient.maritalStatus.coding', None)

detail_dfs = dict(sorted(detail_dfs.items(), key=lambda x: x[0]))
for df in detail_dfs:
    # Drop nan rows from the dataframes
    detail_dfs[df].dropna(how='all', inplace=True)
    # Drop duplicates from the dataframes
    detail_dfs[df].drop_duplicates(inplace=True)

    # Cast best types for the columns
    detail_dfs[df] = detail_dfs[df].convert_dtypes()
    print(f"\nResource type: {df}")
    detail_dfs[df].info(memory_usage=False)


Resource type: AllergyIntolerance
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   clinicalStatus  278 non-null    string
 1   type            278 non-null    string
 2   category        278 non-null    string
 3   criticality     278 non-null    string
 4   assertedDate    278 non-null    object
 5   display         278 non-null    string
 6   code            278 non-null    string
 7   patient_id      278 non-null    string
dtypes: object(1), string(7)
Resource type: CarePlan
<class 'pandas.core.frame.DataFrame'>
Index: 2096 entries, 0 to 3052
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   status           2096 non-null   string
 1   period.start     2096 non-null   string
 2   period.end       1096 non-null   string
 3   activity_status  2096 non-null   stri